In [4]:
import pandas as pd
# import re

df = pd.read_excel("D:/Prep/TechTFQ/Python/PAN+Card+Validation+in+PYTHON+-+Scripts/PAN Card Validation in PYTHON - Scripts/US_Validation_Dataset.xlsx")


In [5]:
len(df)

1020

### Removing rows where both columns are null


In [6]:
df = df.dropna(subset=["SSN","DLN"], how="all").copy()

In [7]:
df["DLN"] = df["DLN"].astype(str).str.strip().str.upper()

0       M6380176
1          KOBRV
2       X6047962
3       DCQDBWNK
4       R0483424
          ...   
1010    X1472344
1011    R3999359
1013    X0046403
1015    U9563333
1019    D6098908
Name: DLN, Length: 804, dtype: object

In [ ]:
# Social Security Number (SSN)

# Format: AAA-GG-SSSS

# Exactly 9 digits (with or without dashes).

# Cannot start with 000, 666, or 9.

# Group GG cannot be 00.

# Serial SSSS cannot be 0000.

# Example Valid: 123-45-6789

# Example Invalid: 000-12-3456, 666-45-6789, 12345678.

In [8]:
df = df.drop_duplicates(subset=["SSN"],keep="first")

In [9]:
df = df.drop_duplicates(subset=["DLN"],keep="first")

In [10]:
len(df)

804

In [11]:
df["SSN"] = df["SSN"].astype(str).str.replace("-","")

In [12]:
# SSN Length Check 
def length_check(ssn):
    if len(ssn) != 9:
        return False
    return True

In [13]:
#  Format: AAA-GG-SSSS
# AAA is not 000 or 666
def first_three(ssn):
    ssn = str(ssn)
    ch = ssn[:3]
    if ch == '000' or ch == '666' or ssn[0] == '9':
        return False
    return True
    

In [31]:
def second_two(ssn):
    ssn = str(ssn)
    ch = ssn[3:5]
    if ch == '00':
        return False
    return True

In [1]:
def last_four(ssn):
    ssn = str(ssn)
    ch = ssn[5:10]
    if ch == '0000':
        return False
    return True

In [38]:
# to check format for Driver's licence, we use regular expression
import re
def dln_format_check(dln):
    if not re.fullmatch(r'^[A-Z][0-9]{7}', dln):
        return False
    return True

In [32]:
def is_valid_ssn(ssn):
    if len(ssn) != 9:
        return False
    if not first_three(ssn):
        return False
    if not second_two(ssn):
        return False
    if not last_four(ssn):
        return False
    return True

In [37]:
# df["SSN_Status"] = df["SSN"].apply(lamda x: "Valid SSN" if is_valid_ssn(x) else "Invalid SSN")

True

In [42]:
def get_ssn_info(x):
    if is_valid_ssn(x):
        return pd.Series(["Valid SSN", f"{str(x)[0:3]}-{str(x)[3:5]}-{str(x)[5:9]}"])
    else:
        return pd.Series(["Invalid SSN",""])
    
df[["SSN_Status","Formatted_SSN"]] = df["SSN"].apply(get_ssn_info)

In [43]:
df

,SSN,DLN,SSN_Status,Formatted_SSN
0,000123456,M6380176,Invalid SSN,
1,686467986,KOBRV,Valid SSN,686-46-7986
2,qk9Otdbq3qh,X6047962,Invalid SSN,
3,491327341,DCQDBWNK,Valid SSN,491-32-7341
4,nan,R0483424,Invalid SSN,
...,...,...,...,...
1010,83193254,X1472344,Invalid SSN,
1011,123689027,R3999359,Invalid SSN,
1013,123364746,X0046403,Valid SSN,123-36-4746
1015,798712818,U9563333,Valid SSN,798-71-2818


In [44]:
df["DLN_Status"] = df["DLN"].apply(lambda x: "Valid DLN" if dln_format_check(x) else "Invalid DLN")

In [45]:
df

,SSN,DLN,SSN_Status,Formatted_SSN,DLN_Status
0,000123456,M6380176,Invalid SSN,,Valid DLN
1,686467986,KOBRV,Valid SSN,686-46-7986,Invalid DLN
2,qk9Otdbq3qh,X6047962,Invalid SSN,,Valid DLN
3,491327341,DCQDBWNK,Valid SSN,491-32-7341,Invalid DLN
4,nan,R0483424,Invalid SSN,,Valid DLN
...,...,...,...,...,...
1010,83193254,X1472344,Invalid SSN,,Valid DLN
1011,123689027,R3999359,Invalid SSN,,Valid DLN
1013,123364746,X0046403,Valid SSN,123-36-4746,Valid DLN
1015,798712818,U9563333,Valid SSN,798-71-2818,Valid DLN


# Summary Report

In [48]:

# Total valid and Invalid SSNs
valid_invalid_ssn = df.groupby("SSN_Status").size()


# Total valid  and Invalid DLNs

valid_invalid_dln = df.groupby("DLN_Status").size()





# Total Missing/Incomplete Records

In [57]:
# Total missing/incomplete records
missing_records = df[
    (df["SSN"].isna()) |
    (df["DLN"].isna()) |
    (df["SSN_Status"] == "Invalid SSN") |
    (df["DLN_Status"] == "Invalid_DLN")
]

print("Total missing or incomplete records:", len(missing_records))


Total missing or incomplete records: 244


# Writing results to new excel file


In [58]:
with pd.ExcelWriter("SSN_DLN_Validation_Results.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Validation_Results", index=False)
    missing_records.to_excel(writer, sheet_name="Missing_or_Invalid", index=False)